In [1]:
# coding: utf-8
import re 
import pandas as pd
import requests
import random
import json
from urllib import parse
from urllib import request
import time
from datetime import datetime
from datetime import timedelta
header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Connection':'keep-alive'
}
cookies = '__mta=147569288.1539613293789.1539613310187.1539616722700.3; v=3; \
        _lxsdk_cuid=166781a789ec8-0139751cb2ee6e-8383268-144000-166781a789fc8; \
        __mta=147569288.1539613293789.1539613293789.1539613310187.2; uuid_n_v=v1; \
        iuuid=9AF20AC0D08D11E89D508D0D63C93F71D10FE6A2B71B406D800371E2CD9160FE; webp=true; ci=96%2C%E6%B5%8E%E5%8D%97; \
        _lxsdk=9AF20AC0D08D11E89D508D0D63C93F71D10FE6A2B71B406D800371E2CD9160FE; _lxsdk_s=166784ecee0-b1a-f0b-22%7C%7C2'
cookie = {}
for line in cookies.split(';'):
    name, value = cookies.strip().split('=', 1)
    cookie[name] = value

def get_id_by_name(name):
    url = 'http://maoyan.com/query?'
    dict1 ={'kw': name}
    url_data = parse.urlencode(dict1) #unlencode()将字典{k1:v1,k2:v2}转化为k1=v1&k2=v2
    html = requests.get(url=url+url_data, cookies=cookie,headers=header).content.decode('utf-8')
    movie_item = re.findall(r'data-val=\"{movieid:(.*?)}">', html)
    if len(movie_item)==0:
        return -1
    else:
        return movie_item[0]
    
def get_release_time_by_name(name):
    id = get_id_by_name(name)
    if id == -1:
        return None
    else:
        url = 'http://maoyan.com/films/'+get_id_by_name(name)
#         print(url)
        html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
        s = r'<li class=\"ellipsis\">(.*?)映<\/li>'
        release_time = re.findall(s, html)
        return release_time[0][0:10]#只截取时间 如 2018-09-30 
def get_comments_continue(name,start_time):
    comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])
    release_time = str(get_release_time_by_name(name))
    end_time =  release_time+' 00:00:00'
    count=0
    pages =15
    
    url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time
    html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
    data = json.loads(html)['cmts']
    pages=len(data)
    start_time = data[14]['startTime']  # 获得末尾评论的时间
    start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
    start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
    
    while pages ==15:  
        try:
            time.sleep(0.2)
            url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time.replace(' ', '%20')
            print(str(count)+"  "+url)
            count = count+1
            html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
            data = json.loads(html)['cmts']
            pages=len(data)

            for item in data:
                if item.get('cityName')==None or item.get('time')==None or item.get('score')==None or item.get('content')==None or item.get('nick')==None:
                    print("jump")
                else:  
                    try:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':item['gender']},
                                                       ignore_index=True)
                    except:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':0},
                                                       ignore_index=True)
            comments.to_csv('all_'+name+'.csv',index=False,mode='a', header=False,encoding="utf_8_sig")
#             print(comments)
            start_time = data[14]['startTime']  # 获得末尾评论的时间
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
            start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
            comments.drop(comments.index,inplace=True)#清空comments   

        except Exception as e:
            time.sleep(0.5)
            print(e)
            continue

In [15]:
def get_comments_all(name):
    comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])
    start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # 获取当前时间，从当前时间向前获取
    release_time = str(get_release_time_by_name(name))
    end_time =  release_time+' 00:00:00'
    comments.to_csv('all_'+name+'.csv',index=False,encoding="utf_8_sig")
    count=0
    pages =15
    
    while pages ==15:  
        try:
#             time.sleep(0.2)
            
            url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time.replace(' ', '%20')
            print(str(count)+"  "+url)
            count = count+1
            html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
            data = json.loads(html)['cmts']
            pages=len(data)

            for item in data:
                if item.get('cityName')==None or item.get('time')==None or item.get('score')==None or item.get('content')==None or item.get('nick')==None:
                    print("jump")
                else:  
                    try:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':item['gender']},
                                                       ignore_index=True)
                    except:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':0},
                                                       ignore_index=True)
            comments.to_csv('all_'+name+'.csv',index=False,mode='a', header=False,encoding="utf_8_sig")
            
            start_time = data[14]['startTime']  # 获得末尾评论的时间
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
            start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
            
            comments.drop(comments.index,inplace=True)#清空comments   

        except Exception as e:
            time.sleep(0.5)
            print(e)
            continue

In [17]:
get_comments_all("影") #9月30上映

0  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-20%2000:53:08
1  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-20%2000:11:32
2  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2023:51:26
3  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2023:38:14
4  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2023:08:34
5  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2022:57:33
6  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2022:35:25
7  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2022:15:02
8  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2021:55:46
9  http://m.maoyan.com/mmdb/comments/movie/1203437.json

KeyboardInterrupt: 

In [19]:
get_comments_continue("影",'2018-10-19%2018:30:55')

0  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2018:20:16
1  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2018:04:31
2  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2017:47:10
3  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2017:21:22
4  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2016:58:54
5  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2016:33:12
6  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2016:02:28
7  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2015:49:57
8  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-19%2015:20:49
9  http://m.maoyan.com/mmdb/comments/movie/1203437.json

78  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2022:46:22
79  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2022:26:00
80  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2022:06:45
81  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2021:56:56
82  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2021:42:00
83  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2021:26:39
84  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2021:02:36
85  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2020:40:02
86  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-17%2020:23:53
87  http://m.maoyan.com/mmdb/comments/movie/12

155  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2015:54:54
156  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2015:38:34
157  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2015:23:41
158  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2015:06:34
159  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2014:48:29
160  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2014:16:48
161  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2013:48:14
162  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2013:23:10
163  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-16%2013:00:11
164  http://m.maoyan.com/mmdb/comment

232  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2008:09:29
233  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2007:17:34
234  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2002:46:59
235  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2001:35:44
236  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2000:59:01
237  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2000:29:14
238  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-15%2000:06:03
239  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2023:54:21
240  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2023:44:18
241  http://m.maoyan.com/mmdb/comment

309  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2013:45:41
310  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2013:37:07
311  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2013:24:50
312  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2013:14:21
313  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2013:08:24
314  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2013:00:51
315  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2012:49:53
316  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2012:38:35
317  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-10-14%2012:25:41
318  http://m.maoyan.com/mmdb/comment

...

3662  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:40:36
3663  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:38:51
3664  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:37:13
3665  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:35:50
3666  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:33:07
3667  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:32:02
3668  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:30:30
3669  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:28:18
3670  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2016:26:55
3671  http://m.maoyan.com/mm

3738  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2014:11:53
3739  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2014:09:03
3740  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2014:06:35
3741  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2014:04:04
3742  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2014:00:38
3743  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2013:58:05
3744  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2013:55:58
3745  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2013:52:32
3746  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-30%2013:49:26
3747  http://m.maoyan.com/mm

3814  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-28%2018:38:23
3815  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-28%2016:49:56
3816  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-28%2011:17:39
3817  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-27%2021:47:29
3818  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-27%2002:04:45
3819  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-25%2022:16:00
3820  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-25%2010:19:24
3821  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-24%2019:26:14
3822  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-09-24%2019:04:17
3823  http://m.maoyan.com/mm

3890  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-26%2015:15:32
3891  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2023:09:45
3892  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:59:01
3893  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:47:20
3894  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:38:06
3895  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:30:49
3896  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:20:42
3897  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:12:09
3898  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-03-22%2022:06:19
3899  http://m.maoyan.com/mm

3966  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-26%2019:32:42
3967  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-26%2015:00:08
3968  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-25%2018:42:30
3969  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-25%2018:12:32
3970  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-23%2018:15:53
3971  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-23%2017:50:43
3972  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-23%2017:24:49
3973  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-12%2020:35:33
3974  http://m.maoyan.com/mmdb/comments/movie/1203437.json?_v_=yes&offset=0&startTime=2018-02-12%2020:18:01
3975  http://m.maoyan.com/mm

In [5]:
get_comments_continue("夏洛特烦恼",'2015-10-05%2019:29:15')

0  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:28:28
jump
jump
1  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:27:22
jump
jump
jump
jump
2  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:26:36
jump
jump
jump
jump
jump
jump
jump
jump
3  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:25:55
jump
4  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:25:05
jump
jump
5  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:24:26
jump
jump
6  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:23:33
jump
jump
jump
jump
7  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2019:22:52
jump
jump
jump
jump
jump
jump
8  http://m.maoyan.com/

jump
68  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:39:55
jump
jump
jump
69  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:39:22
jump
jump
70  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:38:39
jump
jump
jump
71  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:38:01
jump
jump
jump
jump
jump
72  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:37:28
jump
jump
jump
jump
73  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:36:33
jump
jump
jump
jump
jump
74  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:35:54
jump
jump
jump
75  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2018:35:11
jump
jump
jump
jump
76  http:/

jump
jump
jump
jump
jump
136  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:54:06
jump
jump
jump
jump
137  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:53:12
jump
jump
138  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:52:38
jump
jump
139  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:51:40
jump
jump
jump
140  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:50:51
jump
jump
jump
141  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:50:10
jump
jump
jump
142  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:49:35
jump
143  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:48:41
jump
jump
jump
144  http://m.maoyan.c

jump
jump
jump
jump
jump
204  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:06:27
jump
jump
jump
jump
205  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:05:40
jump
jump
jump
206  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:05:05
jump
jump
jump
207  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:04:30
jump
jump
208  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:03:45
jump
209  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:03:10
jump
jump
jump
jump
210  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:02:34
jump
jump
jump
211  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2017:01:55
jump
jump
jump
jump
jump
ju

jump
jump
jump
jump
271  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:16:38
jump
jump
jump
272  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:16:06
jump
jump
jump
273  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:15:27
jump
jump
jump
jump
jump
274  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:14:53
jump
jump
jump
jump
275  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:13:20
jump
jump
jump
jump
jump
jump
jump
276  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:12:06
jump
jump
277  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-05%2016:11:04
jump
jump
jump
jump
jump
278  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-10-0

...

jump
jump
jump
4015  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2016:24:23
jump
jump
jump
jump
4016  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2016:21:10
jump
jump
jump
jump
jump
4017  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2016:14:36
jump
jump
jump
jump
jump
4018  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2016:09:58
jump
4019  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2016:04:14
jump
jump
4020  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2016:01:35
jump
4021  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2015:57:44
jump
jump
4022  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2015:52:50
jump
jump
jump
jump
4023  htt

jump
jump
4082  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2001:35:40
jump
jump
jump
jump
jump
jump
4083  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2000:52:12
jump
jump
4084  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-30%2000:26:59
jump
jump
jump
4085  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-29%2023:53:00
jump
jump
jump
4086  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-29%2023:24:39
4087  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-29%2022:53:57
jump
jump
jump
jump
4088  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-29%2022:29:16
4089  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-29%2021:55:34
jump
jump
4090  http://m.maoyan.com/mmdb/com

jump
jump
jump
4154  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2012:41:28
jump
4155  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2012:23:46
jump
4156  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2012:12:50
jump
jump
4157  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2011:58:48
4158  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2011:46:59
jump
4159  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2011:32:46
jump
4160  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2011:24:09
4161  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%2011:13:15
4162  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-28%201

jump
jump
jump
4223  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2022:00:07
jump
jump
jump
jump
4224  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2021:56:36
jump
jump
jump
jump
jump
4225  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2021:52:38
jump
jump
jump
4226  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2021:48:59
jump
jump
jump
jump
jump
jump
jump
jump
4227  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2021:46:41
jump
jump
jump
jump
jump
4228  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2021:42:50
jump
jump
jump
jump
jump
jump
jump
4229  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2021:38:59
jump
jump
jump
jump
jump
jump
jump
4230  http://m.maoyan.com/mmdb/comments/m

jump
jump
jump
jump
4285  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2017:41:44
jump
jump
jump
4286  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2017:31:05
jump
jump
jump
jump
jump
jump
jump
jump
jump
4287  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2017:22:20
jump
jump
jump
jump
jump
jump
4288  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2017:15:28
jump
jump
jump
4289  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2017:09:17
jump
jump
jump
4290  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2017:01:28
jump
jump
jump
4291  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%2015:47:54
4292  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2015-09-27%